In [1]:
def read_word(f):
    """f is open and seeked to the length byte at the 
    start of a word, read that word and return it (decoded as SHIFT-JIS)
    advances f to the next length"""
    length = f.read(1)[0]
    #print(length)
    b = f.read(length)
    w = b.decode('shift-jis')
    #print(w)
    assert(f.read(3) == b'\x00\x00\x00')
    assert(w)
    assert(not '\x00' in w)
    return w

def read_words(f):
    """f is open and seeked to the wordset length byte
    returns a list of words in that set of words or [] if it's the end"""
    length = f.peek(1)[0] * 2
    if length == 0:
        return False
    else:
        f.seek(4,1)
    return [read_word(f=f) for _ in range(length)]

In [2]:
def read_wordlist(f):
    wordlist = []

    last = read_words(f=f)
    while last:
        wordlist.append(last)
        try:
            last = read_words(f=f)
        except IndexError:
            last = False

    return wordlist

In [3]:
def find_next_section(f, sep=27):
    # this could be cleaner but the files are short enough to just brute force this
    count = 0
    n = 0
    while True:
        s = f.peek(8)[:8]
        n += 1
        #if n % 100 == 0:
        #if count > 1:
        #    print(f'{f.tell()}, {s}, ({count})')
        if len(s) == 0: return False
        elif s[0] == 0x00:
            count += 1
        elif count >= sep:
            return True
        else:
            count = 0
        f.seek(1, 1)

def find_section(f, which='wordlists'):
    # tries to parse blocks of data as wordlists until all are found in the file
    # there is either one or two lists of keys and a list of lists of words
    f.seek(0)
    words = []
    lists = []
    while True:
        if not find_next_section(f=f): break
        try:
            posn = f.tell()
            words = read_wordlist(f=f)
        except:
            pass
        else:
            if words and isinstance(words[0], list):
                lists.append((posn, words))
    # last section is always the wordlists
    return lists[-1]
        
def find_info(f):
    # returns (Genre, Title, Artist)
    f.seek(0)
    find_next_section(f, sep=8)
    return read_word(f), read_word(f), read_word(f)

In [4]:
def pre_words_post(f):
    # file f split into three sections
    # middle section could be replaced by the result of buildWordlists
    offset, _ = find_section(f, which='wordlists')
    f.seek(0)
    pre = f.read(offset)[:offset]
    f.seek(offset)
    read_wordlist(f)
    size = f.tell() - offset
    
    post = f.read()
    
    f.seek(offset)
    wordbytes = f.read(size)[:size]
    
    return pre, wordbytes, post

In [5]:
def buildWordlists(wordlists):
    # returns bytes for given wordlists
    b = bytearray()
    for n, wordlist in enumerate(wordlists):
        if len(wordlist) % 2 == 1:
            raise ValueError(f'{n}th wordlist invalid. must have even number of words. has {len(wordlist)}: {wordlist}')
        b.append(len(wordlist) // 2)
        b.extend([0, 0, 0])
        for word in wordlist:
            encoded = word.encode('shift-jis')
            b.append(len(encoded))
            b.extend(encoded)
            b.extend([0, 0, 0])
    return bytes(b)

In [6]:
files = [
    # pop'n easy first song
    'charts/N01.BMD',
    # hard classic
    'charts/E10.BMD',
    # how about beatmania?
    'charts/~09_NahaGacho.bmd',
    # and beatmania best
    'charts/E10B.bmd',
]

In [7]:
with open(files[0], 'rb') as f:
    contents = f.read()
    pre, wbytes, post = pre_words_post(f)

from io import BufferedReader, BytesIO
with BufferedReader(BytesIO(wbytes)) as f:
    w = read_wordlist(f)

pre+buildWordlists(w)+post == contents

True

#### Note:
Lists alternate between displayed forms and hiragana forms. Be careful to keep the number of wordlists the same and give each wordlist an even number of words.
They don't need to have the same number of words and the game seems to randomly pick words from
each list in sequence for the typing sections.

I would recommend testing words ingame as some kana (notably v's) don't work and romaji don't work either.



Below are a few wordlists from the game:

In [8]:
with open(files[0], 'rb') as f:
    print(find_info(f))
    offset, wordlists = find_section(f=f)
wordlists

('DANCE', 'Hi-Tekno', 'Hi-Tekno')


[['大人', 'おとな', '楽器', 'がっき', '涙', 'なみだ', '音', 'おと'],
 ['検査', 'けんさ', 'テスト', 'てすと', '煙草', 'たばこ', '針', 'はり'],
 ['襟', 'えり', '花', 'はな', '猫', 'ねこ', 'スリル', 'すりる'],
 ['穴', 'あな', 'ケーキ', 'けーき', '空', 'そら', '闇', 'やみ'],
 ['光', 'ひかり', '飛ぶ', 'とぶ', '鳥', 'とり', '曇り', 'くもり'],
 ['晴れ', 'はれ', '風', 'かぜ', 'キッズ', 'きっず', 'カバー', 'かばー'],
 ['彼', 'かれ', 'マル', 'まる', 'バツ', 'ばつ'],
 ['ピンチ', 'ぴんち', '嘘', 'うそ', '靴', 'くつ'],
 ['帽子', 'ぼうし', '脳', 'のう', '今', 'いま'],
 ['夏季', 'かき', '冬期', 'とうき', 'グミ', 'ぐみ'],
 ['リボン', 'りぼん', 'ベル', 'べる', '朝', 'あさ'],
 ['昼', 'ひる', '夜', 'よる', '昨日', 'きのう'],
 ['今日', 'きょう', '愛', 'あい', '雪', 'ゆき'],
 ['雨', 'あめ', '赤', 'あか', '青', 'あお'],
 ['黄色', 'きいろ', '黒', 'くろ', '白', 'しろ'],
 ['緑', 'みどり', 'シャツ', 'しゃつ', '街', 'まち'],
 ['オーラ', 'おーら', 'ソフト', 'そふと', 'ハード', 'はーど'],
 ['メモ', 'めも', '留守', 'るす', '子機', 'こき']]

In [9]:
with open(files[1], 'rb') as f:
    print(find_info(f))
    offset, wordlists = find_section(f=f)
wordlists

('CLASSIC 2', 'R.C.', 'Waldeus von dovjak')


[['脈絡のない会話',
  'みゃくらくのないかいわ',
  '大御所登場',
  'おおごしょとうじょう',
  '将来を見据える',
  'しょうらいをみすえる',
  '社会的影響',
  'しゃかいてきえいきょう'],
 ['中央分離帯',
  'ちゅうおうぶんりたい',
  '二十一世紀',
  'にじゅういっせいき',
  '忘れ難い出来事',
  'わすれがたいできごと',
  '理想郷を求める',
  'りそうきょうをもとめる',
  'カルチャーショック',
  'かるちゃーしょっく'],
 ['映像編集',
  'えいぞうへんしゅう',
  '異常現象',
  'いじょうげんしょう',
  '構造的欠陥',
  'こうぞうてきけっかん',
  '超能力者',
  'ちょうのうりょくしゃ',
  '宇宙開発時代',
  'うちゅうかいはつじだい'],
 ['蒸気機関車',
  'じょうききかんしゃ',
  '会場設営',
  'かいじょうせつえい',
  '軌道衛星上',
  'きどうえいせいじょう',
  '太平洋高気圧',
  'たいへいようこうきあつ',
  '公開討論会',
  'こうかいとうろんかい'],
 ['小笠原諸島',
  'おがさわらしょとう',
  'ハイジャック発生',
  'はいじゃっくはっせい',
  '見直しを求める',
  'みなおしをもとめる',
  '地震予知連絡会',
  'じしんよちれんらくかい'],
 ['シンガーソングライター',
  'しんがーそんぐらいたー',
  'イベント開催中',
  'いべんとかいさいちゅう',
  'バイオテクノロジー',
  'ばいおてくのろじー',
  '鳴り物入りのスタート',
  'なりものいりのすたーと'],
 ['米国西海岸',
  'べいこくにしかいがん',
  '速乾性マニキュア',
  'そっかんせいまにきゅあ',
  '新生活のスタート',
  'しんせいかつのすたーと',
  '小説の挿し絵',
  'しょうせつのさしえ'],
 ['誕生日プレゼント',
  'たんじょうびぷれぜんと',
  'オープンキャンパス',
  'おーぷんきゃんぱす',
  'モーニングサービス',
  'もーにんぐさーびす',
  '社会的立場',
  'しゃかいてきたちば

In [10]:
with open(files[2], 'rb') as f:
    print(find_info(f))
    offset, wordlists = find_section(f=f)
wordlists

('?????', 'NhahaGachooon', 'Hayachiya Shinpey')


[['アーイェーオーイェー',
  'あーいぇーおーいぇー',
  'アイマイミーユーユアユー',
  'あいまいみーゆーゆあゆー',
  '熱いビートを刻み込め',
  'あついびーとをきざみこめ',
  'ありおりはべりいまそかり',
  'ありおりはべりいまそかり',
  '犬も歩けば棒に当たる',
  'いぬもあるけばぼうにあたる',
  'エブリバディーさいこー',
  'えぶりばでぃーさいこー',
  'がちょってがちょーん',
  'がちょってがちょーん',
  '壁に耳あり障子に目あり',
  'かべにみみありしょうじにめあり',
  '華麗に決めるスクラッチ',
  'かれいにきめるすくらっち',
  'ギャラリー達の視線くぎづけ',
  'ぎゃらりーたちのしせんくぎづけ',
  'キレがあるのにコクがある',
  'きれがあるのにこくがある',
  'クールなプレイで勝負',
  'くーるなぷれいでしょうぶ',
  'クラブシーンでナンバーワン',
  'くらぶしーんでなんばーわん',
  '心も体もリフレッシュ',
  'こころもからだもりふれっしゅ',
  '渋谷界隈のクラブで',
  'しぶやかいわいのくらぶで',
  'タイニーケーイカモーン',
  'たいにーけーいかもーん',
  '旅は道連れ世は情け',
  'たびはみちづれよはなさけ',
  'ちりも積もれば山となる',
  'ちりもつもればやまとなる',
  'ディープなギャグセンス',
  'でぃーぷなぎゃぐせんす',
  '飛んで火に入る夏の虫',
  'とんでひにいるなつのむし',
  'ナハナハナハナハナハ',
  'なはなはなはなはなは'],
 ['人気ナンバーワンですわ',
  'にんきなんばーわんですわ',
  '能ある鷹はつめを隠す',
  'のうあるたかはつめをかくす',
  'リーチイッパツツモ',
  'りーちいっぱつつも',
  'ああ言えばこう言う',
  'ああいえばこういう',
  '当たり前田のクラッカー',
  'あたりまえだのくらっかー',
  'あっみだくじーあっみだくじー',
  'あっみだくじーあっみだくじー',
  '奇妙奇天烈摩訶不思議',
  'きみょうきてれつまかふしぎ',
  'こりゃまた失礼しました',
  'こりゃまたしつれ

In [11]:
with open(files[3], 'rb') as f:
    print(find_info(f))
    offset, wordlists = find_section(f=f)
wordlists

('GABBAH', 'HELL SCAPER', 'L.E.D LIGHT-G')


[['鐘が鳴るなり法隆寺',
  'かねがなるなりほうりゅうじ',
  '就職活動熱風',
  'しゅうしょくかつどうねっぷう',
  'ドラマチック練習試合',
  'どらまちっくれんしゅうじあい',
  'ひと夏のアヴァンチュール',
  'ひとなつのあう゛ぁんちゅーる',
  '東名高速渋滞',
  'とうめいこうそくじゅうたい'],
 ['ホームメイドクッキング雑誌',
  'ほーむめいどくっきんぐざっし',
  '大海原の小さな家',
  'おおうなばらのちいさないえ',
  'マスカルポーネクリームチーズ',
  'ますかるぽーねくりーむちーず',
  '二人三脚徒競走',
  'ににんさんきゃくときょうそう',
  '魅力ある登場人物',
  'みりょくあるとうじょうじんぶつ'],
 ['人面魚発見レーダー',
  'じんめんぎょはっけんれーだー',
  'サーカスの曲乗りライオン',
  'さーかすのきょくのりらいおん',
  '獣道を走るライダー',
  'けものみちをはしるらいだー',
  'ツベルクリン反応検査',
  'つべるくりんはんのうけんさ',
  'サンプルクレンジングクリーム',
  'さんぷるくれんじんぐくりーむ'],
 ['目覚まし時計の長い針',
  'めざましどけいのながいはり',
  'サマーバーゲンセール開催',
  'さまーばーげんせーるかいさい',
  '名刺の渡し方マニュアル',
  'めいしのわたしかたまにゅある',
  '国際社会から孤立する',
  'こくさいしゃかいからこりつする'],
 ['選手生命を脅かす',
  'せんしゅせいめいをおびやかす',
  '四季折々の食卓',
  'しきおりおりのしょくたく',
  '厳しい修行に耐える',
  'きびしいしゅぎょうにたえる',
  'シンクロナイズドスイミング',
  'しんくろないずどすいみんぐ'],
 ['スキューバダイビングの基礎',
  'すきゅーばだいびんぐのきそ',
  '応答メッセージ再生',
  'おうとうめっせーじさいせい',
  '平均気温の上昇',
  'へいきんきおんのじょうしょう',
  '事件は現場で起こっている',
  'じけんはげんばでおこっている'],
 ['お肌のお手入れを欠かさない',
  'おはだのおていれをかかさ

In [12]:
# make a new file new_fname with orig_fname's 
# word lists replaced with new_words
def make_new_from(orig_fname, new_fname, new_words):
    with open(orig_fname, 'rb') as f:
        contents = f.read()
        pre, wbytes, post = pre_words_post(f)
        offset, wordlists = find_section(f=f)
    
    assert(len(wordlists) == len(new_words))
    
    new_wordlists = buildWordlists(new_words)
    with open(new_fname, "wb") as new:
        new.write(pre)
        new.write(new_wordlists)
        new.write(post)

make a chart whose wordlists are just katakana alphabet to see what the game does with it

In [13]:
hiragana_list = [
["あいうえお", "あいうえお"],
["かきくけこ", "かきくけこ"],
["さしすせそ", "さしすせそ"],
["たちつてと", "たちつてと"],
["なにぬねの", "なにぬねの"],
["はひふへほ", "はひふへほ"],
["まみむめも", "まみむめも"],
["やゆよ", "やゆよ"],
["わ", "わ"],
["らりるれろ", "らりるれろ"],
["ゐゑをん", "ゐゑをん"],
["がぎぐげご", "がぎぐげご"],
["ざじずぜぞ", "ざじずぜぞ"],
["だぢづでど", "だぢづでど"],
["ばびぶべぼ", "ばびぶべぼ"],
["ぱぴぷぺぽ", "ぱぴぷぺぽ"],
["ぁぃぅぇぉっ", "ぁぃぅぇぉっ"],
["ゃゅょゎ", "ゃゅょゎ"]
]

katakana_list = [
["アイウエオ", "アイウエオ"],
["カキクケコ", "カキクケコ"],
["サシスセソ", "サシスセソ"],
["タチツテト", "タチツテト"],
["ナニヌネノ", "ナニヌネノ"],
["ハヒフヘホ", "ハヒフヘホ"],
["マミムメモ", "マミムメモ"],
["ヤユヨワ", "ヤユヨワ"],
["ラリルレロ", "ラリルレロ"],
["ヰヱヲンヴ", "ヰヱヲンヴ"],
["ガギグゲゴ", "ガギグゲゴ"],
["ザジズゼゾ", "ザジズゼゾ"],
["ダヂヅデド", "ダヂヅデド"],
["バビブベボ", "バビブベボ"],
["パピプペポ", "パピプペポ"],
["ァィゥェォッ", "ァィゥェォッ"],
["ャュョヮ", "ャュョヮ"],
["ヵヶ", "ヵヶ"]
]


make_new_from(files[0], "out/katakana.bmd", katakana_list)